<a href="https://colab.research.google.com/github/JLaura011/Business-Analytics/blob/master/Sesi%C3%B3n_5_Sistemas_de_Recomendaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Sesión 5: Sistemas de Recomendación**



<img src="https://www.grupolambda.com.pe/wp-content/uploads/2020/03/home-logo.png" alt="drawing" width="300"/>
<br>

 ##### Docente: Jorge Brian Alarcon Flores 

##**Caso: Datos de evaluaciones y etiquetas de películas**

In [ ]:
!pip install surprise

In [ ]:
#importamos librerias
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = None

from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

import warnings; warnings.simplefilter('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## **Sistema de recomendación basado en contenido**

**Limitación del modelo de popularidad** <br>
Si lo haces a nivel cliente, puede funcionar, pero si lo haces de manera general, dará la misma recomendación a todos, independientemente del interés personal del usuario. <br>

_Por ejemplo: _ Si una persona que ama las películas románticas mirara las 15 películas románticas principales, y probablemente no le gustaría la mayoría de las películas mencionadas. Entonces, debía ir un paso más allá y mirar las listas de películas por género, todavía no recibiría las recomendaciones interesantes. Digamos que ama * Dilwale Dulhania Le Jayenge *, * Mi nombre es Khan * y * Kabhi Khushi Kabhi Gham *. Probablemente podemos suponer que ama al actor Shahrukh Khan y al director Karan Johar. Incluso si tuviera que acceder a la lista de romance más importante, no los encontraría en la lista.

Por lo tanto, construyamos un modelo que calcule la similitud entre películas en función de ciertas métricas y sugiera películas que sean más similares a una película en particular que le haya gustado a un usuario. Para eso tenemos que considerar los metadatos (o contenido), por lo tanto, también se conoce como **Filtrado basado en contenido**.

Dos recomendaciones basadas en contenido se implementan en función de diferentes contenidos:
1. Basado en la descripción (contenido: resúmenes de películas y lemas)
2. Basado en metadatos (contenido: reparto de películas, equipo, palabras clave y género)

**Nota**: se utiliza un pequeño conjunto de datos de películas debido a la limitación de la potencia informática disponible para mí.

In [ ]:
#Importamos la data
url = 'https://raw.githubusercontent.com/JBrianAlicorp/Business-Analytics/master/movies_metadata.csv'
m_df = pd.read_csv(url,encoding='latin1')
m_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",30/10/1995,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",15/12/1995,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.712900,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",22/12/1995,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",22/12/1995,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",10/02/1995,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [ ]:
url = 'https://raw.githubusercontent.com/JBrianAlicorp/Business-Analytics/master/links_small.csv'
small_mdf = pd.read_csv(url,encoding='latin1')
small_mdf.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [ ]:
small_mdf = small_mdf[small_mdf['tmdbId'].notnull()]['tmdbId'].astype('int')

Antes de extraer un pequeño conjunto de datos, debemos asegurarnos de que la columna de ID de nuestro marco de datos principal esté limpia y sea de tipo entero. Para hacer esto, intentemos realizar una conversión entera de nuestros ID y, si se produce una excepción, reemplazaremos el ID con NaN. Luego procederemos a eliminar estas filas de nuestro marco de datos.

In [ ]:
m_df=m_df.dropna()


In [ ]:
m_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
9,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,14.686036,/5c0ovjT41KnYIHYuF4AWsTe3sKh.jpg,"[{'name': 'United Artists', 'id': 60}, {'name'...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",16/11/1995,352194034.0,130.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,GoldenEye,False,6.6,1194.0
68,False,"{'id': 43563, 'name': 'Friday Collection', 'po...",3500000,"[{'id': 35, 'name': 'Comedy'}]",http://www.newline.com/properties/friday.html,10634,tt0113118,en,Friday,Craig and Smokey are two guys in Los Angeles h...,14.569650,/kkoGT4bnK5oGiNbzGGcXzrXMSxn.jpg,"[{'name': 'New Line Cinema', 'id': 12}]","[{'iso_3166_1': 'US', 'name': 'United States o...",26/04/1995,28215918.0,91.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A lot can go down between thursday and saturda...,Friday,False,7.0,513.0
69,False,"{'id': 10924, 'name': 'From Dusk Till Dawn Col...",19000000,"[{'id': 27, 'name': 'Horror'}, {'id': 28, 'nam...",http://www.miramax.com/movie/from-dusk-till-dawn/,755,tt0116367,en,From Dusk Till Dawn,Seth Gecko and his younger brother Richard are...,15.339153,/ce0d4kM5KxT4x1Oq8JkSuNYevri.jpg,"[{'name': 'A Band Apart', 'id': 59}, {'name': ...","[{'iso_3166_1': 'US', 'name': 'United States o...",19/01/1996,25836616.0,108.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,One night is all that stands between them and ...,From Dusk Till Dawn,False,6.9,1644.0
153,False,"{'id': 439053, 'name': 'Brooklyn Cigar Store C...",2000000,"[{'id': 35, 'name': 'Comedy'}]",http://miramax.com/movie/blue-in-the-face/,5894,tt0112541,en,Blue in the Face,"Auggie runs a small tobacco shop in Brooklyn, ...",11.528147,/qa9yf3OlfXbYBb0af0mpawGB6TI.jpg,"[{'name': 'Miramax Films', 'id': 14}, {'name':...","[{'iso_3166_1': 'US', 'name': 'United States o...",15/09/1995,1275000.0,83.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Welcome to the planet Brooklyn.,Blue in the Face,False,6.8,28.0
178,False,"{'id': 286162, 'name': 'Power Rangers Collecti...",15000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.powerrangers.com/,9070,tt0113820,en,Mighty Morphin Power Rangers: The Movie,Power up with six incredible teens who out-man...,7.024227,/A3ijhraMN0tvpDnPoyVP7NulkSr.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_...",30/06/1995,66000000.0,92.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Power Is On!,Mighty Morphin Power Rangers: The Movie,False,5.2,153.0


In [ ]:
m_df['id'] = m_df['id'].astype('int')

In [ ]:
sm_df = m_df[m_df['id'].isin(small_mdf)]
sm_df.shape

(447, 24)


**447** películas disponibles en nuestro conjunto de datos 

### 1. Descripción basada en recomendación

In [ ]:
sm_df['tagline'] = sm_df['tagline'].fillna('')
sm_df['description'] = sm_df['overview'] + sm_df['tagline']
sm_df['description'] = sm_df['description'].fillna('')

#### Creamos la matriz TF-IDF

In [ ]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(sm_df['description'])
tfidf_matrix.shape

(447, 19329)


#### Similitud de coseno

La similitud de coseno se utiliza para calcular una cantidad numérica que denota la similitud entre dos películas. Matemáticamente, se define de la siguiente manera:

$ coseno (x, y) = \ frac {x. y ^ \ intercal} {|| x ||. || y ||} $

Como se utiliza el vectorizador TF-IDF, el cálculo del producto de puntos nos dará directamente el puntaje de similitud de coseno. Por lo tanto, se utiliza ** linear_kernel ** de sklearn en lugar de cosine_similarities, ya que es mucho más rápido.

In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
cosine_sim[0]

array([1.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.00696763, 0.00759807, 0.01252464,
       0.01163625, 0.        , 0.        , 0.00840636, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.01022136, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.01406922, 0.        , 0.02136082, 0.        , 0.        ,
       0.01532349, 0.        , 0.        , 0.        , 0.01008677,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.00816239, 0.        , 0.        , 0.        , 0.        ,
       0.06599254, 0.        , 0.        , 0.        , 0.01132739,
       0.        , 0.        , 0.        , 0.        , 0.04011309,
       0.01536705, 0.03462974, 0.04202035, 0.06305757, 0.06410072,
       0.08148224, 0.0456975 , 0.01470343, 0.02375263, 0.     

Ahora tenemos una matriz de similitud de coseno por pares para todas las películas de nuestro conjunto de datos. El siguiente paso es escribir una función que devuelva las 50 películas más similares según el puntaje de similitud de coseno.

In [ ]:
sm_df = sm_df.reset_index()
titles = sm_df['title']
indices = pd.Series(sm_df.index, index=sm_df['title'])

In [ ]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:51]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

Probemos ahora y obtenga las 10 mejores recomendaciones para algunas películas.

In [ ]:
get_recommendations('The Godfather').head(10)

308    Elite Squad: The Enemy Within
47           The Godfather: Part III
292                          Machete
116            House of 1000 Corpses
42                      Halloween II
403                       The Raid 2
262                     Halloween II
437                        Furious 7
172              The Hills Have Eyes
11                        Home Alone
Name: title, dtype: object

In [ ]:
get_recommendations('The Dark Knight').head(10)

340                 The Dark Knight Rises
342    Sherlock Holmes: A Game of Shadows
154                         Batman Begins
446    Batman v Superman: Dawn of Justice
382                       Despicable Me 2
40                    Tomorrow Never Dies
104                              Blade II
295       Wall Street: Money Never Sleeps
416          Sin City: A Dame to Kill For
71                            Thunderball
Name: title, dtype: object

Vemos que para **The Dark Knight**, nuestro sistema puede identificarlo como una película de Batman y, posteriormente, recomendar otras películas de Batman como sus principales recomendaciones. Pero desafortunadamente, eso es todo lo que este sistema puede hacer en este momento. Esto no es de mucha utilidad para la mayoría de las personas, ya que no tiene en cuenta características muy importantes como el reparto, el equipo, el director y el género, que determinan la calificación y la popularidad de una película. Alguien a quien le gustó **The Dark Knight** probablemente le guste más por el actor principal y odiaría **Batman y Superman** y cualquier otra película de calidad inferior en la franquicia de Batman.

Por lo tanto, vamos a utilizar metadatos mucho más sugerentes que **Descripción general** y **Lema**. En la siguiente subsección, construyamos un recomendador más sofisticado que tenga en cuenta **género**, **palabras clave**, **reparto** y **tripulación** en consideración.

### 2.  Recomendación basada en metadata

In [ ]:
url = 'https://raw.githubusercontent.com/JBrianAlicorp/Business-Analytics/master/keywords.csv'
keywords = pd.read_csv(url,encoding='latin1')
keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [ ]:
keywords=keywords.dropna()
keywords.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46419 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46419 non-null  int64 
 1   keywords  46419 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.1+ MB


In [ ]:
keywords['id'] = keywords['id'].astype('int')
m_df['id'] = m_df['id'].astype('int')
m_df.shape

(603, 24)

In [ ]:
# Cruzamos con nuestra data principal
m_df = m_df.merge(keywords, on='id')

In [ ]:
sm_df = m_df[m_df['id'].isin(small_mdf)]
sm_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,keywords
0,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,14.686036,/5c0ovjT41KnYIHYuF4AWsTe3sKh.jpg,"[{'name': 'United Artists', 'id': 60}, {'name'...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",16/11/1995,352194034.0,130.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,GoldenEye,False,6.6,1194.0,"[{'id': 701, 'name': 'cuba'}, {'id': 769, 'nam..."
1,False,"{'id': 43563, 'name': 'Friday Collection', 'po...",3500000,"[{'id': 35, 'name': 'Comedy'}]",http://www.newline.com/properties/friday.html,10634,tt0113118,en,Friday,Craig and Smokey are two guys in Los Angeles h...,14.569650,/kkoGT4bnK5oGiNbzGGcXzrXMSxn.jpg,"[{'name': 'New Line Cinema', 'id': 12}]","[{'iso_3166_1': 'US', 'name': 'United States o...",26/04/1995,28215918.0,91.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A lot can go down between thursday and saturda...,Friday,False,7.0,513.0,"[{'id': 897, 'name': 'rap music'}, {'id': 970,..."
2,False,"{'id': 10924, 'name': 'From Dusk Till Dawn Col...",19000000,"[{'id': 27, 'name': 'Horror'}, {'id': 28, 'nam...",http://www.miramax.com/movie/from-dusk-till-dawn/,755,tt0116367,en,From Dusk Till Dawn,Seth Gecko and his younger brother Richard are...,15.339153,/ce0d4kM5KxT4x1Oq8JkSuNYevri.jpg,"[{'name': 'A Band Apart', 'id': 59}, {'name': ...","[{'iso_3166_1': 'US', 'name': 'United States o...",19/01/1996,25836616.0,108.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,One night is all that stands between them and ...,From Dusk Till Dawn,False,6.9,1644.0,"[{'id': 246, 'name': 'dancing'}, {'id': 380, '..."
3,False,"{'id': 439053, 'name': 'Brooklyn Cigar Store C...",2000000,"[{'id': 35, 'name': 'Comedy'}]",http://miramax.com/movie/blue-in-the-face/,5894,tt0112541,en,Blue in the Face,"Auggie runs a small tobacco shop in Brooklyn, ...",11.528147,/qa9yf3OlfXbYBb0af0mpawGB6TI.jpg,"[{'name': 'Miramax Films', 'id': 14}, {'name':...","[{'iso_3166_1': 'US', 'name': 'United States o...",15/09/1995,1275000.0,83.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Welcome to the planet Brooklyn.,Blue in the Face,False,6.8,28.0,"[{'id': 919, 'name': 'smoking'}, {'id': 1901, ..."
4,False,"{'id': 286162, 'name': 'Power Rangers Collecti...",15000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.powerrangers.com/,9070,tt0113820,en,Mighty Morphin Power Rangers: The Movie,Power up with six incredible teens who out-man...,7.024227,/A3ijhraMN0tvpDnPoyVP7NulkSr.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_...",30/06/1995,66000000.0,92.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Power Is On!,Mighty Morphin Power Rangers: The Movie,False,5.2,153.0,"[{'id': 10988, 'name': 'based on tv series'}, ..."


In [ ]:
sm_df['keywords'] = sm_df['keywords'].apply(literal_eval)

In [ ]:
sm_df['keywords'] = sm_df['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

#### Palabras clave

Haremos una pequeña cantidad de preprocesamiento de nuestras palabras clave antes de utilizarlas. Como primer paso, calculamos los conteos frecuentes de cada palabra clave que aparece en el conjunto de datos.

In [ ]:
s = sm_df.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

In [ ]:
s = s.value_counts()
s[:5]

duringcreditsstinger    82
aftercreditsstinger     45
3d                      41
sequel                  40
dystopia                36
Name: keyword, dtype: int64

Las palabras clave aparecen en frecuencias que van del 1 al 610. No utilizamos las palabras clave que aparecen una sola vez. Por lo tanto, estos se pueden eliminar de forma segura. Finalmente, convertiremos cada palabra a su raíz para que palabras como * Perros * y * Perro * se consideren iguales.

In [ ]:
s = s[s > 1]

In [ ]:
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [ ]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [ ]:
sm_df['keywords'] = sm_df['keywords'].apply(filter_keywords)
sm_df['keywords'] = sm_df['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
sm_df['keywords'] = sm_df['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [ ]:
sm_df['soup'] = sm_df['keywords']
sm_df['soup'] = sm_df['soup'].apply(lambda x: ' '.join(x))

In [ ]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(sm_df['soup'])

In [ ]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [ ]:
sm_df = sm_df.reset_index()
titles = sm_df['title']
indices = pd.Series(sm_df.index, index=sm_df['title'])

Reutilizaremos la función get_recommendations que habíamos escrito anteriormente. Dado que nuestros puntajes de similitud de coseno han cambiado, esperamos que nos dé resultados diferentes (y probablemente mejores). Revisemos nuevamente **The Dark Knight** y veamos qué recomendaciones recibo esta vez.

In [ ]:
get_recommendations('The Dark Knight').head(10)

341                 The Dark Knight Rises
155                         Batman Begins
447    Batman v Superman: Dawn of Justice
379                          Man of Steel
416          Teenage Mutant Ninja Turtles
432            Captain America: Civil War
319                    X-Men: First Class
281                            Iron Man 2
430                               Ant-Man
144                        Blade: Trinity
Name: title, dtype: object

Las recomendaciones parecen haber reconocido otras películas que coinciden con palabras claves y las han puesto como las mejores recomendaciones. La persona disfrutó viendo **The Dark Knight**, así como algunos de los otros en la lista, incluyendo **Batman Begins**, **Iron Man 2**.

## **Sistema de recomendación : Collaborative Filtering**


**Limitación de la recomendación basada en contenido**:
- Solo es capaz de sugerir películas que son * cercanas * a una película determinada. Es decir, no es capaz de captar interés y proporcionar recomendaciones en todos los géneros.

- No captura el interés personal y los prejuicios de un usuario. Cualquier persona que busque recomendaciones basadas en una película en el modelo recibirá las mismas recomendaciones para esa película, independientemente de quién sea.

Por lo tanto, en esta sección, utilizaremos una técnica llamada **Filtrado colaborativo** para hacer recomendaciones a los observadores de películas. El Filtrado Colaborativo se basa en la idea de que los usuarios similares a mí pueden usarse para predecir cuánto me gustará un producto o servicio en particular que esos usuarios hayan usado / experimentado, pero yo no.

Utilizaremos la librería **Surprise** que utilizó algoritmos extremadamente potentes como **Descomposición de valor singular (SVD)** para minimizar RMSE (error cuadrático medio) y ofrecer excelentes recomendaciones.

In [ ]:
reader = Reader()

In [ ]:
url = 'https://raw.githubusercontent.com/JBrianAlicorp/Business-Analytics/master/ratings_small.csv'
ratings = pd.read_csv(url,encoding='latin1')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [ ]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
from surprise.model_selection import KFold
kf = KFold(n_splits=5)
kf.split(data)

<generator object KFold.split at 0x7fa743ff25c8>

In [ ]:
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'])

{'fit_time': (4.605312347412109,
  4.59843897819519,
  4.5615479946136475,
  4.606761455535889,
  4.594202041625977),
 'test_mae': array([0.69138548, 0.68982983, 0.69629392, 0.68958017, 0.68494708]),
 'test_rmse': array([0.89690039, 0.89855754, 0.90322069, 0.89506992, 0.88948517]),
 'test_time': (0.1451709270477295,
  0.2210252285003662,
  0.13037848472595215,
  0.22027587890625,
  0.12544775009155273)}

Obtenemos un **error cuadrático medio** de alrededor de 0.8965 que es más que suficiente en este momento. Ahora entrenemos en el conjunto de datos y lleguemos a las predicciones.

In [ ]:
trainset = data.build_full_trainset()
svd.fit(trainset)

Seleccionemos el usuario 1 y verifiquemos las calificaciones que ha otorgado.



In [ ]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [ ]:
svd.predict(1, 302, 3)

Prediction(uid=1, iid=302, r_ui=3, est=2.8511903403434813, details={'was_impossible': False})

Para películas con ID 302, obtenemos una predicción estimada de **2.74**. Una característica sorprendente de este sistema de recomendación es que no le importa qué es la película (o qué contiene). Funciona únicamente sobre la base de una ID de película asignada e intenta predecir las clasificaciones en función de cómo los otros usuarios han predicho la película

## **Sistema de recomendación combinada**

Creamos un simple recomendador híbrido que reúne las técnicas que hemos implementado en los motores basados ​​en contenido y basados ​​en filtros colaborativos. Así es como funcionará:

* **Entrada:** ID de usuario y el título de una película
* **Salida:** Películas similares ordenadas según las calificaciones esperadas por ese usuario en particular.

In [ ]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [ ]:
url = 'https://raw.githubusercontent.com/JBrianAlicorp/Business-Analytics/master/links_small.csv'
small_mdf = pd.read_csv(url,encoding='latin1')
id_map = small_mdf[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(sm_df[['title', 'id']], on='id').set_index('title')
#id_map = id_map.set_index('tmdbId')

In [ ]:
indices_map = id_map.set_index('id')

In [ ]:
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    #print(idx)
    movie_id = id_map.loc[title]['movieId']
    
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = sm_df.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [ ]:
hybrid(1, 'Avatar')

,title,vote_count,vote_average,id,est
324,Harry Potter and the Deathly Hallows: Part 2,6141.0,7.9,12445,3.381820
403,Captain America: The Winter Soldier,5881.0,7.6,100402,3.191784
30,Alien,4564.0,7.9,348,2.924754
271,Alvin and the Chipmunks: The Squeakquel,688.0,5.3,23398,2.850663
329,A Very Harold & Kumar Christmas,335.0,6.0,55465,2.770786
347,Journey 2: The Mysterious Island,1050.0,5.8,72545,2.741988
366,Piranha 3DD,319.0,4.1,71668,2.690870
291,Piranha 3D,599.0,5.3,43593,2.630086
442,The Hunger Games: Mockingjay - Part 2,4117.0,6.6,131634,2.622316
243,My Bloody Valentine,303.0,5.3,14435,2.582299


In [ ]:
hybrid(500, 'Avatar')

,title,vote_count,vote_average,id,est
324,Harry Potter and the Deathly Hallows: Part 2,6141.0,7.9,12445,3.606442
393,Riddick,2066.0,6.2,87421,3.453597
347,Journey 2: The Mysterious Island,1050.0,5.8,72545,3.244327
360,Prometheus,5152.0,6.3,70981,3.152952
217,Aliens vs Predator: Requiem,758.0,4.9,440,3.145095
291,Piranha 3D,599.0,5.3,43593,3.129854
389,Percy Jackson: Sea of Monsters,1685.0,5.9,76285,3.099430
366,Piranha 3DD,319.0,4.1,71668,3.061531
403,Captain America: The Winter Soldier,5881.0,7.6,100402,3.043339
442,The Hunger Games: Mockingjay - Part 2,4117.0,6.6,131634,3.018635


Vemos que para nuestro recomendante híbrido, obtenemos diferentes recomendaciones para diferentes usuarios, aunque la película es la misma. Por lo tanto, nuestras recomendaciones son más personalizadas y personalizadas para usuarios particulares.


## **Conclusión**

En este notebook hemos construido 3 motores de recomendación diferentes basados ​​en diferentes ideas y algoritmos. Son los siguientes:

1. **Recomendador basado en contenido:** Construimos dos motores basados ​​en contenido; uno que tomó la visión general de la película y los lemas como entrada y el otro que tomó metadatos como las palabras clave para elaborar predicciones. También utilizamos un filtro simple para dar mayor preferencia a las películas con más votos y calificaciones más altas.
2. **Filtrado colaborativo:** Utilizamos la poderosa Biblioteca Surprise para construir un filtro colaborativo basado en la descomposición de un solo valor. El RMSE obtenido fue inferior a 1 y el motor dio clasificaciones estimadas para un usuario y una película determinados.
3. **Motor híbrido:** Reunimos ideas del contenido y el filtrado colaborativo para crear un motor que ofreciera sugerencias de películas a un usuario en particular en función de las clasificaciones estimadas que había calculado internamente para ese usuario.
